# Data Cleaning

## **Import package**

In [1]:
import pandas as pd
import numpy as np
import re
# from symspellpy import SymSpell, Verbosity
from collections import Counter
# from underthesea import word_tokenize
import json
from google.colab import drive
from difflib import get_close_matches
from bs4 import BeautifulSoup
import html


In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


## **Read data**

In [3]:
df = pd.read_json(
    "/content/drive/MyDrive/statlearning/ner_labeled_data_full.jsonl",
    lines=True,
    encoding="utf-8"
)

In [4]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 36,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 291...",https://vietstock.vn/2025/05/thi-truong-chung-...


In [7]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 41,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 114...",https://vietstock.vn/2025/05/thi-truong-chung-...


In [8]:
df.loc[0, 'text'][41:44]

'HAG'

## **Loại bỏ các kí tự vô nghĩa trong text**

In [10]:
def clean_text(text: str) -> str:
    """
    1. Bỏ escape HTML entities (&nbsp;, &amp; …)
    2. Tháo hết tag HTML còn sót (nếu parse thiếu)
    3. Xóa ký tự control, non‑breaking space, zero‑width space
    4. Chỉ giữ:
       - chữ (bao gồm dấu tiếng Việt)
       - số
       - dấu câu .,;:!?()-/%
       - khoảng trắng
    5. Gom nhiều khoảng trắng thành 1 và strip hai đầu
    """
    if text is None:
        return ""

    # 1. Unescape HTML entities
    s = html.unescape(text)

    # 2. Tháo tag HTML
    s = BeautifulSoup(s, "html.parser").get_text(separator=" ")

    # 3. Xóa các ký tự control và non‑printing
    s = s.replace("\xa0", " ")   # non‑breaking space
    s = s.replace("\u200b", "")  # zero‑width space
    s = re.sub(r"[\r\n\t]+", " ", s)

    # 4. Loại bỏ ký tự lạ, chỉ giữ chữ, số, dấu câu, space
    s = re.sub(r"[^0-9A-Za-zÀ-ỹ.,;:!?\-\(\)\[\]/% ]+", " ", s)

    # 5. Gom khoảng trắng
    s = re.sub(r"\s+", " ", s).strip()
    return s

In [11]:
content = ">>> *SGR kết thúc sớm đợt chào bán 20 triệu cp, Chủ tịchPhạm Thumua gần 9.9 triệu cp."
print(clean_text(content))

SGR kết thúc sớm đợt chào bán 20 triệu cp, Chủ tịchPhạm Thumua gần 9.9 triệu cp.


In [12]:
df['text'] = df['text'].apply(clean_text)

In [13]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 41,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 114...",https://vietstock.vn/2025/05/thi-truong-chung-...


## **Đánh số lại các entities**

In [14]:
def update_entity_spans(text: str, entities: list) -> list:
    """
    Cập nhật lại chỉ số start/end cho entities dựa trên text hiện tại.
    - text: chuỗi đã clean
    - entities: danh sách dict có keys 'text' và 'type' (có thể kèm các trường khác, nhưng start/end sẽ được ghi đè)
    Trả về list mới với đầy đủ 'text','type','start','end'.
    """
    new_entities = []
    cursor = 0  # vị trí bắt đầu tìm kiếm trong text

    for ent in entities:
        ent_text = ent["text"]
        # tìm lần xuất hiện của ent_text trong text, bắt đầu từ cursor
        idx = text.find(ent_text, cursor)
        if idx == -1:
            # không tìm thấy nữa, bỏ qua hoặc bạn có thể log idx để kiểm tra
            continue
        start = idx
        end = idx + len(ent_text)

        # xây dựng bản ghi mới, giữ lại mọi trường khác trong ent
        new_ent = ent.copy()
        new_ent["start"] = start
        new_ent["end"]   = end
        new_entities.append(new_ent)

        # di chuyển con trỏ để khỏi match lại cùng một vị trí
        cursor = end

    return new_entities


In [15]:
# Áp dụng cho từng dòng
df["entities"] = df.apply(
    lambda row: update_entity_spans(row["text"], row["entities"]),
    axis=1
)

In [16]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 41,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 114...",https://vietstock.vn/2025/05/thi-truong-chung-...


## **Sửa stock_code và tên company**

In [17]:
stock_list = []
with open("/content/drive/MyDrive/statlearning/all_vn_stock_codes.txt", "r", encoding="utf-8") as f:
    stock_list = [line.strip().upper() for line in f if line.strip()]

In [18]:
def build_common_company_names(df, top_n=500):
    company_names = []
    for entities in df["entities"]:
        for ent in entities:
            if ent["type"] == "COMPANY":
                company_names.append(ent["text"].strip())
    most_common = Counter(company_names).most_common(top_n)
    return [name for name, _ in most_common]

In [19]:
company_list = build_common_company_names(df)

In [32]:
company_list

['Fed',
 'UBCKNN',
 'NHNN',
 'KBSV',
 'VNDIRECT',
 'CTCK',
 'Dragon Capital',
 'ACBS',
 'MSCI',
 'Bộ Tài chính',
 'KIS',
 'Aseansc',
 'DNSE',
 'Vingroup',
 'Yuanta',
 'BSC',
 'VCBS',
 'FTSE Russell',
 'SCIC',
 'VinaCapital',
 'FTSE',
 'VietinBank',
 'PYN Elite',
 'Vietcombank',
 'Vietstock',
 'SSV',
 'BIDV',
 'VDSC',
 'Vietcap',
 'VSDC',
 'Ủy ban Chứng khoán Nhà nước',
 'IFC',
 'Pomina',
 'Masan',
 'Novaland',
 'Masan Consumer',
 'Berkshire Hathaway',
 'Coteccons',
 'Vinamilk',
 'TPS',
 'F88',
 'HDI Global SE',
 'UOB',
 'SJC',
 'VNG',
 'SSIR',
 'SSIAM',
 'DAS',
 'Novagroup',
 'Sacombank',
 'HAGL',
 'CTCK BIDV',
 'CTCK KB Việt Nam',
 'FPTS',
 'BETA',
 'Vinpearl',
 'Vinatex',
 'FLC',
 'Hòa Phát',
 'Hải An',
 'Vinhomes',
 'SK Group',
 'Mirae Asset',
 'Vinalines',
 'VietstockFinance',
 'Ngân hàng Nhà nước',
 'BlackRock',
 'CTCK Asean',
 'Công ty Điện lực An Phú Đông',
 'Tập đoàn Masan',
 'Apple',
 'Hưng Thịnh Land',
 'Viconship',
 'Huatai',
 'HSC',
 'BVSC',
 'VPS',
 'Beta',
 'SSI Research'

In [35]:
with open("/content/drive/MyDrive/statlearningRoot/list_company.txt", "w", encoding="utf-8") as f:
  for item in company_list:
    f.write(f"{item}\n")


In [21]:
def safe_get_span(text, value):
    try:
        start = text.index(value)
        end = start + len(value)
        return start, end
    except ValueError:
        return -1, -1


In [23]:
content = "VC"
matched = get_close_matches(content, stock_list, n=1, cutoff=0.7)
matched

['VTC']

In [24]:
def clean_entities_and_text(text, entities, stock_list, company_list, cutoff=0.7):
    new_text = text
    temp_entities = []

    offset = 0  # Để tính sự thay đổi độ dài sau khi thay thế từ

    for ent in entities:
        ent_text = ent["text"]
        ent_type = ent["type"]
        ent_start = ent.get("start")
        ent_end = ent.get("end")

        if ent_start is None or ent_end is None:
            # ent_start, ent_end = safe_get_span(new_text, ent_text)
            # if ent_start == -1:
            #     temp_entities.append(ent)
                continue  # Không tìm thấy thì bỏ qua xử lý sửa

        # Lấy đoạn gốc trong text hiện tại
        current_text = new_text[ent_start + offset: ent_end + offset]

        if ent_type == "STOCK":
            corrected = ent_text.upper()
            if corrected in stock_list:
                # Thay đúng vị trí trong text
                # "Chung ta coo thanh" -> "Chung ta co thanh"
                new_text = new_text[:ent_start + offset] + corrected + new_text[ent_end + offset:]
                delta = len(corrected) - (ent_end - ent_start)
                temp_entities.append({
                    "text": corrected,
                    "type": ent_type,
                    "start": ent_start + offset,
                    "end": ent_start + offset + len(corrected)
                })
                offset += delta
            else:
                # Giữ nguyên nhưng vẫn gán lại vị trí
                temp_entities.append({
                    "text": ent_text,
                    "type": ent_type,
                    "start": ent_start + offset,
                    "end": ent_end + offset
                })

        elif ent_type == "COMPANY":
            matched = get_close_matches(ent_text, company_list, n=1, cutoff=cutoff)
            corrected = matched[0] if matched else ent_text
            new_text = new_text[:ent_start + offset] + corrected + new_text[ent_end + offset:]
            delta = len(corrected) - (ent_end - ent_start)
            temp_entities.append({
                "text": corrected,
                "type": ent_type,
                "start": ent_start + offset,
                "end": ent_start + offset + len(corrected)
            })
            offset += delta

        else:
            # Giữ nguyên hoàn toàn cho các loại khác
            temp_entities.append({
                "text": ent_text,
                "type": ent_type,
                "start": ent_start + offset,
                "end": ent_end + offset
            })

    return new_text, temp_entities


In [25]:
for i in range(len(df)):
    raw_text = df.at[i, "text"]
    raw_ents = df.at[i, "entities"]

    new_text, new_ents = clean_entities_and_text(raw_text, raw_ents, stock_list, company_list)

    # Ghi đè vào chính cột gốc
    df.at[i, "text"] = new_text
    df.at[i, "entities"] = new_ents


In [26]:
df.head()

,text,entities,source
0,Con gái bầu Đức gom bất thành 4 triệu cp HAG v...,"[{'text': 'HAG', 'type': 'STOCK', 'start': 41,...",https://vietstock.vn/2025/05/con-gai-bau-duc-g...
1,"HOSE tháng 4: Hầu hết chỉ số ngành giảm điểm, ...","[{'text': 'HOSE', 'type': 'STOCK', 'start': 0,...",https://vietstock.vn/2025/05/hose-thang-4-hau-...
2,Đầu tư Phát triển Mỹ Khánh tất toán trước hạn ...,"[{'text': '2.2 ngàn tỷ', 'type': 'NUM', 'start...",https://vietstock.vn/2025/05/dau-tu-phat-trien...
3,Thanh khoản UPC oM tăng 13% trong tháng 4.Thị ...,"[{'text': 'tăng 13%', 'type': 'NUM', 'start': ...",https://vietstock.vn/2025/05/thanh-khoan-upcom...
4,Thị trường chứng quyền tuần 19-23/05/2025: Sắc...,"[{'text': 'SSI', 'type': 'STOCK', 'start': 114...",https://vietstock.vn/2025/05/thi-truong-chung-...


In [28]:
# Biểu thức tìm ký tự “ngoài” tập cho phép
bad_char_pattern = re.compile(r'[^0-9A-Za-zÀ-ỹ.,;:!?\-\(\)\[\]/% ]')
# Tạo mặt nạ (mask) những dòng có ký tự lạ
mask_bad = df['text'].str.contains(bad_char_pattern)

# Danh sách index
bad_indices = df.index[mask_bad].tolist()

print(f"Có {len(bad_indices)} dòng chứa ký tự lạ:", bad_indices)

Có 0 dòng chứa ký tự lạ: []


In [27]:
# import json

with open("/content/drive/MyDrive/statlearningRoot/cleaned_data_full.jsonl", "w", encoding="utf-8") as f:
    for _, row in df.iterrows():
        json.dump({
            "text": row["text"],
            "entities": row["entities"],
            "source" : row['source']
        }, f, ensure_ascii=False)
        f.write("\n")
